# Data engineering

Le but de cette partie est:
- Prendre connaissance des données.
- Nétoyer les données.
- Re-échantilloner les données à une période de 1 minute.

## Initialization

In [ ]:
# std
import glob
import inspect
import json
import os
# external
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import gdown
# local
from utils import missingDF

In [ ]:
# Active la recharge automatique de modules dans un Jupyter Notebook
%load_ext autoreload
# %autoreload 0 : Désactive la recharge automatique.
# %autoreload 1 : Recharge automatiquement les modules importés avec %aimport uniquement.
# %autoreload 2 : Recharge tous les modules (sauf ceux exclus par %aimport), à chaque fois que du code est exécuté.
%autoreload 2

## Paramètres

In [ ]:
dataversion = "data-2310-Yewan-MissingValues-20231106"  # Data version.
limit_oversampling = 30 # propagation of the values in minute for oversampling all-sky status.

In [ ]:
data_directory = os.getcwd() + os.sep  + "data" + os.sep + dataversion  # Directory where we put the data.
path_data_raw = data_directory + os.sep + 'tenerife_2020.csv'  # Path to raw data.
path_label = data_directory + os.sep + 'result.csv' # Path of the label recoverd from all-sky camera.
path_data_1m = data_directory + os.sep + "tenerife_2020_1m.csv" # Path of the output file.
cols_nametypes = [
    ('time', "string"),
    ('status', "string"),
    ('d_ext_temp', 'float32'),
    ('d_humid', 'float32'),
    ('d_rain_rate', 'float32'),
    ('d_wind', 'float32'),
    ('d_wind_dir', 'float32'),
    ('day_r0', 'float32'),
    ('day_see', 'float32'),
    ('day_see_stddev', 'float32'), 
    ('down_ir', 'float32'),
    ('humid', 'float32'),
    ('irrad', 'float32'),
    ('isoplan', 'float32'),
    ('night_r0', 'float32'),
    ('night_see', 'float32'),
    ('press', 'float32'),
    ('pyr_temp', 'float32'),
    ('scint', 'float32'),
    ('sky_temp', 'float32'),
    ('transp', 'float32'),
    ('wat_col_hei', 'float32')
]  # Variables in raw data
cols_names = [i[0] for i in cols_nametypes]
column_types = [i[1] for i in cols_nametypes]

## Fonctions pour facililter le traitement

`missingDF` pour afficher le pourcentage de valeurs manquantes

In [ ]:
print(inspect.getsource(missingDF))

## Données météorologique

### Téléchargement des données

In [ ]:
if not os.path.isdir(data_directory):
    url = "https://drive.google.com/drive/folders/1DLXwAVHCKarucsmaKhQSP786Sws5nWDw"
    gdown.download_folder(url, quiet=False, use_cookies=False, output=data_directory)
else:
    print("Data already downloaded.")

### Vérification que les données on bien était téléchargé

In [ ]:
l_dataset = glob.glob("data" + os.sep + "*" + os.sep + "*.csv")

if len(l_dataset) == 0:
    print("ERREUR: il n'y a pas de données") 
    raise
else:
    for i in l_dataset:
        print(i)

In [ ]:
dtype = dict(zip(cols_names, column_types))
df = pd.read_csv(path_data_raw, usecols=cols_names, dtype=dtype)
#df.replace(0, np.nan, inplace=True)
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
df['time'] = pd.to_datetime(df['time'], unit="ns")

In [ ]:
plt.plot(df.time, df.d_ext_temp)
plt.title("temperature")
plt.show()

In [ ]:
n_raw = len(df)
print(f"Le nombre de ligne du dataset est {n_raw/1000} milles")

On séléctionne un échantillon de donnnées de 5 éléments

In [ ]:
d_month = (df.time.max()-df.time.min()).days/30
print(f"Les données sont réparties sur {d_month:.0f} mois.")

In [ ]:
df.sample(5).T

On regarde de plus prés les valeurs manquantes avec la fonction `missingDF`.

In [ ]:
missingDF(df)

Les valeurs qui nous interesse `night_r0` et `day_r0` on environ $75$% de valeurs manquantes.

### Nétoyage des doublons

In [ ]:
n_dupli = sum(df.duplicated())
print(f"Il y a {n_dupli} ligne dupliqué.")

In [ ]:
df = df[~df.time.duplicated()]

In [ ]:
n_ddupli = len(df)
print(f"Maintenant le dataset contient {n_ddupli/1000} milles lignes.")

### Définition d'un index temporelle

In [ ]:
df.index = df['time']
df.drop('time', axis=1, inplace=True)

In [ ]:
ax = df[['day_r0', "night_r0", "d_ext_temp"]].plot()

### Les valeurs manquantes

Nous allons observer de plus près les valeurs manquantes.

In [ ]:
missing_df = missingDF(df)
print('Dataframe des variables associées leurs taux de valeurs manquantes')
print(missing_df)

Visualisation des données manquantes sur 12 mois.

In [ ]:
cols = sorted(df.columns.tolist())
ax = msno.matrix(df[cols])

Visulisation des données manquates pour le moi de juillet

In [ ]:
cols = sorted(df.columns.tolist())
ax = msno.matrix(df[cols][df.index.month == 7]) # pour juillet

La variable `isoplan` est toujours nulle donc nous allons la retirer.

In [ ]:
df = df.drop(['isoplan'],axis=1)

On considère que les valeurs NaN dans la variable `d_rain_rate corresponde à des jours ou il n'y a pas de pluits

In [ ]:
mask = df.d_rain_rate.isnull()
df.loc[mask, "d_rain_rate"] = 0

### Encodage du status du DIMM

La variable `status` indique l'état opérationnel du DIMM.  

#### Encodage à l'aide d'entiés

Encode l'objet en tant que type énuméré ou variable catégorique.
Cette méthode est utile pour obtenir une représentation numérique d'un tableau lorsque tout ce qui compte est d'identifier des valeurs distinctes.
Cette approche est dynamique, donc même si de nouvelles valeurs de statut sont ajoutées à la colonne à l'avenir, elles se verront automatiquement attribuer des valeurs numériques uniques.
Pour plus d'information voir la documentation: https://pandas.pydata.org/pandas-docs/version/1.5/reference/api/pandas.factorize.html

In [ ]:
numeric_values, unique_statuses = pd.factorize(df['status'], use_na_sentinel=-1)
# na_sentinel=-1 means NaNs are assigned -1, then we add 1 to shift everything so NaNs become 0
df['dimm_numeric'] = numeric_values + 1 

# Create a mapping dictionary
status_mapping = dict(enumerate(unique_statuses))
# Shift dictionary keys by 1 to account for NaN becoming 0
status_mapping = {k+1: v for k, v in status_mapping.items()}
status_mapping[0] = np.nan
print(status_mapping)

In [ ]:
with open('dimm.json', 'w') as f:
    json.dump(status_mapping, f)

#### Encodage avec une matrice de binaire

In [ ]:
df_dimm = pd.get_dummies(df.status.fillna("Unknown")).copy()
df_dimm.columns = ["dimm_" + i.lower().split(" ")[0].lower() for i in df_dimm.columns.tolist()]

In [ ]:
df = pd.merge(df, df_dimm, left_index=True, right_index=True)

On enlève status pour facilter la manipulation des données

In [ ]:
df.drop("status", axis=1, inplace=True)

### Re-échantillonage

In [ ]:
df = df.astype(float)

In [ ]:
df = df.resample('1T').mean()

## Données caméra all-sky

On ajouter les données de notre modèle d'analyse d'image pour la detection d'événement climatique.

Les classes detecté sont les suivantes:

In [ ]:
states_name = {
    "c0": "night", 
    "c1": "sunny", 
    "c2": "cloud", 
    "c3": "fog", 
    "c4": "rain", 
    "c5": "foreign", 
    "c6": "freeze"
}

### Lecture des données

In [ ]:
df_states = pd.read_csv(path_label, usecols=['utc', 'c0', 'c1', 'c2','c3', 'c4', 'c5', 'c6'])
df_states['utc'] = pd.to_datetime(df_states['utc'], unit="ns")
df_states.index = df_states['utc']
df_states.drop('utc', axis=1, inplace=True)
old2new = {}
for i in range(len(states_name)):
    old2new[f"c{i}"] = "cam_" + states_name[f"c{i}"]
df_states.rename(columns=old2new, inplace=True)

display(df_states.index.min())
display(df_states.index.max())

display(df_states.head(5))

display(df_states.describe())

### Netoyage des données

In [ ]:
mask = (df_states.describe().T['unique'] == 2).copy()
col_to_keep = df_states.describe().T['unique'][mask].index.tolist()
df_states = df_states[col_to_keep].copy()
df_states = df_states.astype(float)

### Re-échantillonage

In [ ]:
df_states = df_states.resample("1T").nearest(limit=limit_oversampling)

In [ ]:
df_states.info()

In [ ]:
df_states.describe()

In [ ]:
off = 60*24*10
ax = df_states[off:60*24+off].plot()

### Jointure avec le dataframe principal                                                

In [ ]:
df = df.merge(df_states, left_index=True, right_index=True, how='outer')

In [ ]:
df.head(5).T

In [ ]:
missingDF(df)

## Un dataframe plus facile à manipuler

In [ ]:
df.sample(5).T

In [ ]:
cols = sorted(df.columns.tolist())
ax = msno.matrix(df[cols])

In [ ]:
ax = df["d_ext_temp"].plot()

In [ ]:
df.index.name = "time"
df.to_csv(path_data_1m, index=True)